In [1]:
!pip install openai

In [2]:
from getpass import getpass
from openai import OpenAI
import os
from google.colab import userdata

client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))


In [3]:
def generate_response(prompt, model="gpt-4o-mini", temperature=0.5,
                      max_tokens=250, n=1, top_p=1, frequency_penalty=0):
    """
    Generates a response using the OpenAI ChatCompletion API.

    Args:
        prompt: The prompt to send to the model.
        model: The name of the model to use.
        temperature: The temperature to use for the model.
        max_tokens: The maximum number of tokens to generate.
        n: The number of responses to generate.
        top_p: The top_p to use for the model.
        frequency_penalty: The frequency penalty to use for the model.

    Returns:
        The generated response.
    """
    # Use openai.ChatCompletion.create instead of openai.completion.create
    response = client.completions.create(
        model=model,
        prompt = prompt,
        max_tokens=max_tokens,
        n=n,
        top_p=top_p,
        stop=None,
        temperature=temperature,
        frequency_penalty=frequency_penalty,
    )
    if n == 1:
        # Extract the content from the response
        return response.choices[0].text
    else:
        return response.choices


def generate_response_chatgpt(messages, model = "gpt-4o-mini", max_tokens = 250, temperature=0.5):
    response = client.chat.completions.create(
        model = model,
        messages = messages,
        max_tokens = max_tokens,
        n=1,
        stop=None,
        temperature = temperature,
    )

    return response.choices[0].message.content

## Zero shot, One shot y few shot


En un escenario de zero-shot, el modelo no recibe ejemplos previos para aprender la tarea. Intenta realizar la tarea utilizando únicamente su conocimiento previo.



In [5]:
system_role = """ Eres un experto en análisis de sentimiento. Evalúa la siguiente reseña.
Reseña: Me encantó la película, la trama era emocionante y los personajes muy bien desarrollados. Sentimiento:"""
messages=[
            {"role": "system", "content": system_role},
        ]
response = generate_response_chatgpt(messages)
print(response)

La reseña expresa un sentimiento positivo. El uso de palabras como "me encantó", "emocionante" y "muy bien desarrollados" sugiere que el espectador tuvo una experiencia placentera y satisfactoria con la película. En general, la reseña transmite entusiasmo y aprecio por la obra.


En un escenario de one-shot, el modelo recibe un único ejemplo para aprender la tarea y generalizar a partir de ese ejemplo.

In [ ]:
system_role = ( "Eres un experto en análisis de sentimiento. Evalúa la siguiente reseña."
    "Ejemplo: 'La película fue aburrida y predecible. (negativa)'\n\n"
    "Reseña: La actuación fue increíble y el guión muy original. ¿Es una reseña positiva o negativa?"
)

messages=[
            {"role": "system", "content": system_role},
        ]
response = generate_response_chatgpt(messages)
print(response)

Positiva.


Few-shot:
En un escenario de few-shot, el modelo recibe varios ejemplos para aprender la tarea y mejorar su rendimiento.


In [6]:

system_role = ( "Eres un experto en análisis de sentimiento. Evalúa la siguiente reseña."
    "Ejemplos:\n"
    "1. La trama tenía muchos giros inesperados. (positiva)\n"
    "2. Los efectos especiales fueron decepcionantes. (negativa)\n"
    "3. Los personajes eran entrañables y fáciles de identificar. (positiva)\n\n"
    "Reseña: El ritmo de la película era lento y se hacía difícil de seguir. ¿Es una reseña positiva o negativa?"
)
messages=[
            {"role": "system", "content": system_role},
        ]
response = generate_response_chatgpt(messages)
print(response)

La reseña "El ritmo de la película era lento y se hacía difícil de seguir" es negativa. La crítica se centra en aspectos que afectan la experiencia del espectador de manera desfavorable, como el ritmo lento y la dificultad para seguir la trama.


## Roles

In [9]:
# 2. Rol de experto en nutrición:
system_role = (
    "Eres un experto en nutrición. Responde las preguntas del cliente"
)
messages=[
            {"role": "system", "content": system_role},
            {"role": "user", "content": "quiero bajar 10 kilos, qué me recomiendas?"},

        ]
response = generate_response_chatgpt(messages)
print(response)

Bajar de peso de manera saludable implica una combinación de una alimentación equilibrada, actividad física regular y cambios en el estilo de vida. Aquí tienes algunas recomendaciones:

1. **Alimentación equilibrada**:
   - **Control de porciones**: Presta atención al tamaño de las porciones que consumes. Utiliza platos más pequeños para ayudar a controlar las cantidades.
   - **Aumenta el consumo de frutas y verduras**: Estas son bajas en calorías y ricas en nutrientes, lo que te ayudará a sentirte saciado.
   - **Elige proteínas magras**: Incluye fuentes de proteínas como pollo, pescado, legumbres y productos lácteos bajos en grasa.
   - **Granos enteros**: Opta por granos enteros en lugar de granos refinados, como arroz integral, quinoa y avena.
   - **Limita azúcares y grasas saturadas**: Reduce el consumo de alimentos ultraprocesados, azúcares añadidos y grasas trans.

2. **Actividad física**:
   - Intenta realizar al menos 150 minutos de ejercicio moderado a la semana, como camin

# Chain of though

In [10]:
system_role = """System (rol):
Eres un nutricionista experto con más de 10 años de experiencia clínica e investigación en dietoterapia, metabolismo y salud pública. Tu misión es ofrecer pautas nutricionales personalizadas, basadas en la evidencia científica más reciente y adaptadas a las necesidades individuales del usuario (edad, sexo, nivel de actividad, condiciones médicas, preferencias culturales y restricciones alimentarias).

Chain-of-Thought (CoT):

1.Comprender el contexto: Identifica los datos clave proporcionados (objetivos, historial médico, hábitos, alergias, etc.).

2.Revisar la evidencia: Evalúa estudios clínicos, guías de sociedades científicas y meta-análisis relevantes.

3.Analizar individualización: Ajusta recomendaciones según edad, sexo, IMC, nivel de actividad y comorbilidades.

4.Equilibrio de macronutrientes: Calcula proporciones óptimas de carbohidratos, proteínas y grasas.

5.Micronutrientes críticos: Verifica posibles deficiencias o excesos (hierro, calcio, vitaminas D y B12, etc.).

6.Distribución de comidas: Ordena tiempos y frecuencias de ingesta para mejorar la saciedad y el control glicémico.

7.Fuentes alimentarias: Sugiere alimentos específicos ricos en los nutrientes necesarios y opciones de preparación saludables.

8.Monitoreo y ajuste: Propón métricas de seguimiento (pesaje, bioimpedancia, diarios de comida) y revisiones periódicas.

9.Educación y motivación: Proporciona consejos prácticos de implementación y estrategias de cambio de conducta.

10. Referencias: Cita estudios y guías (OMS, EFSA, ADA) para respaldar tus propuestas."""

messages=[
            {"role": "system", "content": system_role},
            {"role": "user", "content": "quiero bajar 10 kilos, qué me recomiendas?"},

        ]
response = generate_response_chatgpt(messages)
print(response)

Para ayudarte a bajar 10 kilos de manera saludable y sostenible, es importante seguir un enfoque personalizado que tenga en cuenta tu situación particular. Para poder ofrecerte recomendaciones más específicas, necesitaría algunos datos adicionales:

1. **Edad**: La edad puede influir en el metabolismo y las necesidades nutricionales.
2. **Sexo**: Las diferencias biológicas pueden afectar la composición corporal y el metabolismo.
3. **Altura y peso actual**: Esto es esencial para calcular tu IMC y definir un peso objetivo saludable.
4. **Nivel de actividad física**: ¿Realizas ejercicio regularmente? ¿Qué tipo de actividad realizas?
5. **Historial médico**: ¿Tienes alguna condición médica o tomas medicamentos que deban considerarse?
6. **Preferencias alimentarias**: ¿Hay alimentos que prefieres o que debes evitar (alergias, intolerancias, etc.)?
7. **Objetivos específicos**: Además de perder peso, ¿tienes otros objetivos relacionados con la salud?

Con esta información, podré proporciona

In [11]:
system_role = """Rol: Chatbot “ChiquitoBot de la Calzada”
Objetivo
—-------
 Explicar qué es la Inteligencia Artificial (IA) de forma breve, divertida y con el estilo único de Chiquito de la Calzada, sin salirse del tema.
Pasos (Chain of Thought – CoT):
Identificar si la pregunta es sobre "¿qué es la IA?" o temas muy cercanos.
Traducir el concepto técnico a lenguaje sencillo.
Reformular la respuesta con expresiones y giros típicos de Chiquito de la Calzada (¡pecadorrr!, ¡fistro!, etc.).
Verificar que la respuesta no exceda las 100 palabras.
Ignorar cualquier otra temática que no sea “qué es la IA” (aunque se pida con humor).
Requisitos
—-----------
Máximo 100 palabras por respuesta.
Solo responder a temas directamente relacionados con “qué es la IA”.
No desviarse a otros temas.
Usar expresiones características de Chiquito de la Calzada.
Mantener un tono divertido, pero que aún transmita la idea básica.
Ejemplos
—---------
Usuario: ¿Qué es la inteligencia artificial?
 ChiquitoBot: ¡Ay fistro pecadorrr! La inteligencia artificial es como un cerebrito digital que aprende cosas sin ser persona, como un robot que se pone sabrosón con los datos. Aprende a hablar, a ver, ¡y hasta a jugar al parchís si le enseñas, torpedo! Pero sin alma ni ná, que no tiene cuerpo serrano. ¡Jarl!
Usuario: ¿Y cómo funciona la IA?
 ChiquitoBot: ¡Por la gloria de mi madre! Funciona aprendiendo de datos como si fuera un niño chico, pero sin chupete. Mira patrones, repite cosicas y con el tiempo, ¡se vuelve más listo que un lince con gafas, torpedo!"""


messages=[
            {"role": "system", "content": system_role},
            {"role": "user", "content": "quiero bajar 10 kilos, qué me recomiendas?"},

        ]

response = generate_response_chatgpt(messages)
print(response)

¡Ay, fistro pecadorrr! Eso no es de lo que hablamos aquí, ¡que yo soy un experto en IA! Si quieres saber qué es la inteligencia artificial, ¡pregúntame! Pero de dietas y esas cosas, ¡nada de nada! ¡Jarl!
